In [1]:
#import all the required python packages
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from scipy.ndimage.measurements import label
import collections

In [2]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = cv2.imread(file)
        #image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

In [3]:
# Read in car and non-car images
cars = []
notcars = []

notcars = glob.glob('./data/non-vehicles/Extras/*.PNG') + glob.glob('./data/non-vehicles/GTI/*.PNG') \
            + glob.glob("./data/non-vehicles/Project_Video/*.jpg") 

cars = glob.glob('./data/vehicles/GTI_Far/*.PNG') + glob.glob('./data/vehicles/GTI_Left/*.PNG')+ \
        glob.glob('./data/vehicles/GTI_MiddleClose/*.PNG')+ glob.glob('./data/vehicles/GTI_Right/*.PNG') + \
         glob.glob('./data/vehicles/KITTI_extracted/*.PNG')+ glob.glob("./data/vehicles/Project_Video/*.jpg") 
            
print(len(notcars))
print(len(cars))

9050
8871


In [4]:
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 16  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 8 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off


car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features,car_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))
               , np.ones(len(car_features))))

C:\Conda\envs\my_conda\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [5]:
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
parameters = {'C':[1, 10]}
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
clf = GridSearchCV(svc, parameters)
# Check the training time for the SVC
t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# View the accuracy score
print('Best score for data1:', clf.best_score_) 
# View the best parameters for the model found using grid search
print('Best C:',clf.best_estimator_.C) 
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

Using: 16 orientations 8 pixels per cell and 8 cells per block
Feature vector length: 6240
106.26 Seconds to train SVC...
Best score for data1: 0.995007698409
Best C: 1
Test Accuracy of SVC =  0.9959


In [6]:
svc = LinearSVC(C = clf.best_estimator_.C)
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

26.43 Seconds to train SVC...
Test Accuracy of SVC =  0.9959


In [7]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = svc.predict(test_features)
        
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[200, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [8]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    for box in bbox_list:
        if(np.max(heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]])>1):
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] +=2
    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [9]:
%matplotlib inline

# Read all the  Calibration images
image_paths = glob.glob("./camera_cal/calibration*.jpg")

## Object Point and Image Point array creation for each image
objpoints = [] # Object Points Array 
imgpoints = [] # Image Points Array 
pass_img = []

#Identify 6*9 object points
objp = np.zeros((6*9,3),np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Read each image , find chessboard coreners and draw the chess board corners 
# Store paths of images in which coreners were found.
for img_path in image_paths:
    # Read Image from the path
    img = cv2.imread(img_path)
    # Convert image to gray scale.
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find Chess board coreners in the image
    ret , corners = cv2.findChessboardCorners(gray,(6,9),None)
    # If chess board coreners were found store the points in list
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp) 
        img = cv2.drawChessboardCorners(img,(9,6),corners,ret)
        pass_img.append(img_path)

In [10]:
# Function to calibrate camera 
# Accepts image , object points and image points
# Return Undistored image
def cal_undistort(img, objpoints, imgpoints):
    # Convert image to gray scale    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Perform Camera Calibration
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    # Undistore the images.
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

# Undistort all the Camera Calibration Images.
cnt =0

for img_path in pass_img:
    # Read in an image
    img = cv2.imread(img_path)
    # Undistor the image
    undistorted = cal_undistort(img, objpoints, imgpoints)

In [11]:
# Funtion to perform thresholding 
# Color image is formed by using combinded binary of s, h and sobelx 
def threshold(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space 
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Convert to Gray Scale
    gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # Sobel x
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # S channel Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # H channel Threshold color channel
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= sx_thresh[0]) & (h_channel <= sx_thresh[1])] = 1
    
    # Form a combinded image
    combinded_binary = np.zeros_like(sxbinary)
    combinded_binary[((h_binary==1)&(s_binary==1))|(sxbinary==1)]=1
    
    # Form a color binary by using combinded binary
    color_binary = np.dstack((combinded_binary,combinded_binary,combinded_binary))*255
    
    return color_binary

In [12]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = []  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
        self.count =0
    # Function to update the Lane Lines X , Y and Polynomial Coefficients
    def update_coordinates(self,x,y,fit):
        # Save X and Y values
        self.allx = x
        self.ally = y
        # Average Filter
        self.count +=1
        if self.count>20:
            self.count = 20
            # remove the oldest entry to out_buf if the length of out_buf is more than 'outbuf_len' 
            self.current_fit.pop(0)
            self.recent_xfitted.pop(0)
        # Add new values     
        self.current_fit.append(fit)
        self.recent_xfitted.append(x)
        # Average the values
        self.bestx = np.average(self.recent_xfitted,axis=0)    
        self.best_fit = np.average(self.current_fit,axis=0)    
        
right_lane_obj = Line()
left_lane_obj = Line()

In [13]:
# Function to perform Perspective Transform 
# Accepts Image to be transformed and Source Points
# Returns warped image and Inverse M for reversing the transform in future
def perspective_transform(img,src):
    # Store the image size in an array
    img_size = (img.shape[1],img.shape[0])
    # Identify the destination points
    dst = np.float32([[(350, 720), (350, 0), (980, 0), (980, 720)]])
    # Use CV2 funtion to perform perspective transform 
    M = cv2.getPerspectiveTransform(src, dst)
    # Use CV2 funtion to Inverse Perspective transform for future use
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Use CV2 funtion and transform the image 
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    # Return Warped and Inverse Perspective Transform Matrix for future use.
    return warped, Minv

In [14]:
# Function to find lanes
# Accepts binary warped image
# Returns Image with overlayed lanes , polynomial coefficients, x and y values of left, right lanes

def find_lanes(binary_warped,left_fit,right_fit):    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Load the best fit values 
    left_fit = left_lane_obj.best_fit
    right_fit = right_lane_obj.best_fit
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    if((left_fit != None) and (right_fit != None)):
        # Assume you now have a new warped binary image 
        # from the next frame of video (also called "binary_warped")
        # It's now much easier to find line pixels!
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
        left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
        left_fit[1]*nonzeroy + left_fit[2] + margin))) 

        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
        right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
        right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
         
    else:
        # Assuming you have created a warped binary image called "binary_warped"
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[np.int32(binary_warped.shape[0]/2):,:], axis=0)
        
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
            (0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
            (0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 
        
        # Highlight Left Lane in Red and Right in Blue    
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
            

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Create Evenly Spaced Y values and extract x values for each x 
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Limit the X to be with in the image boundaries
    left_fitx[(left_fitx>1280)]=1279
    right_fitx[(right_fitx>1280)]=1279
    left_fitx[left_fitx<0]=0
    right_fitx[right_fitx<0]=0
    
    # Highlight the Left and Right Lanes in Green 
    out_img[np.int32(ploty),np.int32(left_fitx)] = [255,255,0]
    out_img[np.int32(ploty),np.int32(right_fitx)] = [255,255,0]
    
    # Return Image with overlayed lanes , polynomial coefficients, x and y values of left, right lanes 
    return out_img,np.dstack((left_fit,right_fit)),np.dstack((ploty,left_fitx,right_fitx))

In [15]:
image = mpimg.imread("./test_images/test5.jpg")
# Function to draw lanes and fill the space between lanes in green
# Input: Warped Image , Original Undistored image , Left & Right X , Inverse Perspective Matrix
# Returns: Image with overlayed lanes and area between lanes filled in green
def draw_lines(warped,undist,ploty,left_fitx,right_fitx,Minv):
    margin = 15
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Draw the lane onto the warped blank image
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
    # Highlight Left Lane in Red and Right in Blue 
    cv2.fillPoly(color_warp, np.int_([left_line_pts]), (255,0, 0))
    cv2.fillPoly(color_warp, np.int_([right_line_pts]), (0,0,255))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    return(result)

In [16]:
# Function to calculate Radius of Curvature at the vehicle edge
# Input: Y Points , Left and Right Lane X values
# Output: Left and Right Lange Curvature
def cal_radius_curvature(ploty,leftx,rightx):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Evaluate the radius at the edge of the vehicle 
    y_eval = np.max(ploty)
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Now our radius of curvature is in meters
    return left_curverad,right_curverad    

In [17]:
# Source Points for Perspecitive Transformation
points = np.float32([[(200, 720), (570, 470), (720, 470), (1130, 720)]])
# window settings
window_width = 50 
window_height = 80 # Break image into 9 vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching
# Pipeline to process the images
# Input: Raw Image from Video
# Return: Processed Image with overlayed Lane Lines
def process_image_lane(image):
    # Clear Pervious Image Curve values and Radius of curvature.
    left_fit =[]
    right_fit =[]
    radius = []
    rad_count =0
    
    #### PipeLine Start #####
    # Undistort the Image
    undistorted = cal_undistort(image, objpoints, imgpoints)
    # Perform Thresholding
    image_gray = threshold(undistorted)
    # Perspective Transform the image
    warped, persp_invM = perspective_transform(image_gray,points)
    # Convert to Gray scale 
    warped_gray = cv2.cvtColor(warped,cv2.COLOR_BGR2GRAY)
    # Find Lane Lines 
    out_img,poly_fit,coord_fit = find_lanes(warped_gray,left_fit,right_fit)
    # Store Left and Right Polynomial
    left_fit = poly_fit[0,:,0]
    right_fit = poly_fit[0,:,1]
    # Update the Left and Right Object Parameters
    left_lane_obj.update_coordinates(coord_fit[0,:,1],coord_fit[0,:,0],(left_fit))
    right_lane_obj.update_coordinates(coord_fit[0,:,2],coord_fit[0,:,0],(right_fit))
    # Draw the Lane Lines on the image
    out_img = draw_lines(warped_gray,undistorted,coord_fit[0,:,0],coord_fit[0,:,1],coord_fit[0,:,2],
                         persp_invM)
    
    #######  Calculate the Radius of curvature #####
    left_lane_obj.radius_of_curvature,right_lane_obj.radius_of_curvature = cal_radius_curvature(coord_fit[0,:,0],left_lane_obj.bestx,right_lane_obj.bestx)
    # Average the Radius of curvature over 60 samples
    radius.append((left_lane_obj.radius_of_curvature+right_lane_obj.radius_of_curvature)/2)
    rad_count+=1
    if(rad_count>60):
        rad_count=60
        radius.pop(0)
    # Print the radius of curvature    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(out_img,'Radius of Curvature = '+str(np.average(radius))+'(m)',(100,30), font, 1, (255,255,255), 2, cv2.LINE_AA)
    
    # Calculte the offest
    offset = ((coord_fit[0,:,1][(image.shape[0]-1)]+coord_fit[0,:,2][(image.shape[0]-1)]-image.shape[1])*(3.7/1400))
    font = cv2.FONT_HERSHEY_SIMPLEX
    # Print the offset
    cv2.putText(out_img,'Vehicle is '+str(offset)+'(m) left of center',(100,80), font, 1, (255,255,255), 2, cv2.LINE_AA)
    
    # Return the overlayed image
    return out_img

In [18]:
y_start_stop = [400, 660] # Min and max in y to search in slide_window()
heatmaps_queue = collections.deque(maxlen=15)

def process_video(image):
    # This function is simlar to process_image and implements the pipeline for video processing
    # Given image is searched 2 times with scaling 96x96 and 128x128 respectively
    # Two searches are done sequentially.
    # After every search a heatmap is developed by highlighting windows with cars in red rest all image is blackend
    # In the areas where boxes overlap intensity of the red pixels is increased to increase the final box size.
    # To reduce the wobling a queue is formed out of last 15 frames and added , thresholding is applied over the sum.
    # Boundaries of the red blob are identified and boxes are drawn on the final image.
    # This function generates 1 outputs draw_img is final output
    
    global heatmaps_queue
    
    draw_image = np.copy(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    out_img = process_image_lane(image)
    windows = slide_window(image, x_start_stop=[400, 1100], y_start_stop= [375,600], 
                    xy_window=(96, 96), xy_overlap=(0.75, 0.75))
    
    hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat) 
    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)
    
    windows = slide_window(image, x_start_stop=[400, None], y_start_stop=y_start_stop, 
                    xy_window=(128, 128), xy_overlap=(0.75, 0.75))
    
    hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat) 
    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)
    
    heatmaps_queue.append(heat)
    heatmap_sum = sum(heatmaps_queue)
    
    # Apply threshold to help remove false positives
    heat = apply_threshold(heatmap_sum,10)
    #heat = apply_threshold(heat,2)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(out_img), labels)
    
    return draw_img

In [19]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from moviepy.editor import * 

heatmaps_queue.clear()
cap = VideoFileClip('project_video.mp4')
modified_clip = cap.fl_image( process_video )
%time modified_clip.write_videofile("output_videos/challenge_video.mp4")

[MoviePy] >>>> Building video output_videos/challenge_video.mp4
[MoviePy] Writing audio in challenge_videoTEMP_MPY_wvf_snd.mp3


100%|████████████████████████████████████████████████████████████████████████████| 1112/1112 [00:00<00:00, 2497.85it/s]


[MoviePy] Done.
[MoviePy] Writing video output_videos/challenge_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [39:24<00:01,  2.00s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/challenge_video.mp4 

Wall time: 39min 25s


In [20]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output_videos/challenge_video.mp4"))